In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyarrow==15.0.0

In [ ]:
!pip install datasets --quiet

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import ipywidgets as widgets
from IPython.display import display

In [ ]:
home_path = '/content/drive/MyDrive/KMITL/Fourth Year/Project'

file_dropdown = widgets.Dropdown(
    options=[file_name for file_name in os.listdir(home_path) if file_name.endswith('.csv')],
    description='select dataset',
)
display(file_dropdown)

Dropdown(description='select dataset', options=('Full Dataset.csv', 'Test01 Dataset.xlsx - 500.csv'), value='F…

In [ ]:
print(f'selected file: {file_dropdown.value}')
file_path = os.path.join(home_path, file_dropdown.value)
df = pd.read_csv(file_path)

df = df[['News','Symbol','Keyword (Aspect)','Polarity (Opinion)','Sentiment']]
df.head()

selected file: Test01 Dataset.xlsx - 500.csv


,News,Symbol,Keyword (Aspect),Polarity (Opinion),Sentiment
0,TOP ปันผลระหว่างกาล 1.20 บาท ขึ้น XD วันที่ 12...,TOP,ปันผลระหว่างกาล,ขึ้น,Positive
1,"GPSC เผย ""ทิติพงษ์ จุลพรศิริดี"" ลงจาก CFO ""พรร...",GPSC,CFO,ลงจาก,Neutral
2,"PTTEP เผย ""สัมฤทธิ์ สำเนียง"" ลงจาก CFO ""ชนมาศ ...",PTTEP,CFO,ลงจาก,Neutral
3,IIG ยกเลิกเพิ่มทุนขาย Sycamore ชี้ตั้งราคาต่ำไ...,IIG,เพิ่มทุน,ยกเลิก,Neutral
4,IIG ยกเลิกเพิ่มทุนขาย Sycamore ชี้ตั้งราคาต่ำไ...,IIG,ผู้ถือหุ้น,เสียเปรียบ,Negative


In [ ]:
df[df.isnull().any(axis=1)]

,News,Symbol,Keyword (Aspect),Polarity (Opinion),Sentiment
167,SABUY ฟื้น !! | ออฟเรคคอร์ด,SABUY,NaN,ฟื้น,Positive
192,"SC-PR9 บวกดักทาง ""อุ๊งอิ๊ง"" ขึ้นนายกฯ โบรกเกอร...",SC,NaN,บวก,Positive
197,ITD ขาดทุน 1.7 พันล้าน ใน Q2/67 ผู้สอบบัญชีไม่...,ITD,NaN,ขาดทุน,Negative
208,EA ครึ่งปีแรก กำไร 1.4 พันล้าน มั่นใจเคลียร์ดบ...,EA,NaN,กำไร,Positive
272,PTT โชว์งบ Q2/67 แรง 76.4% กำไร 3.5 หมื่นล้าน ...,PTT,NaN,กำไร,Positive
273,AU กำไร Q2/67 พุ่ง 68% ก.ย.นี้ผุด After You แห...,AU,NaN,กำไร,Positive
292,PTTGC พลิกกำไร Q2/67 ที่ 1.8 พันล้าน รับรู้ซื้...,PTTGC,NaN,กำไร,Positive
304,MGI รอดไป !! I ออฟเรคคอร์ด,MGI,NaN,รอด,Neutral
308,BSRC พลิกกำไร 221 ล้านใน Q2/67 โรงกลั่นศรีราชา...,BSRC,NaN,กำไร,Positive
318,TU กำไร Q2/67 ที่ 1.2 พันล้าน เพิ่มเป้ายอดขายป...,TU,NaN,กำไร,Positive


In [ ]:
sent_tag = {
    'positive':'<POS>',
    'negative':'<NEG>',
    'neutral': '<NEU>',
}
SENTIMENT_TAGS = [v for k,v in sent_tag.items()] # ['<POS>','<NEG>','<NEU>']
SPECIAL_TAGS = ['<SYMBOL>','<ASPECT>','<OPINION>'] + SENTIMENT_TAGS

In [ ]:
dataset = pd.DataFrame(columns=['text','label'])
dataset['text'] = df['News']

In [ ]:
def to_label_text(label):
  label_text = ''
  if label.Symbol:
    label_text += f'<SYMBOL> {label.Symbol} '
  if label.Aspect:
    label_text += f'<ASPECT> {label.Aspect} '
  if label.Opinion:
    label_text += f'<OPINION> {label.Opinion} '
  if label.Sentiment:
    label_text += sent_tag[label.Sentiment.lower()]

  return label_text

In [ ]:
print(df.columns)

Index(['News', 'Symbol', 'Keyword (Aspect)', 'Polarity (Opinion)',
       'Sentiment'],
      dtype='object')


In [ ]:
def to_label_text(row): # Changed 'label' to 'row' for clarity
  label_text = ''
  if row['Symbol']: # Accessing values using column names
    label_text += f'<SYMBOL> {row["Symbol"]} '
  if row['Keyword (Aspect)']:
    label_text += f'<ASPECT> {row["Keyword (Aspect)"]} '
  if row['Polarity (Opinion)']:
    label_text += f'<OPINION> {row["Polarity (Opinion)"]} '
  if row['Sentiment']:
    try:  # Attempt to convert to lowercase
      label_text += sent_tag[str(row['Sentiment']).lower()] # Changed to str(row['Sentiment']) to ensure it's a string
    except (KeyError, AttributeError):  # Handle errors if lowercase conversion or key lookup fails
      label_text += ''  # Or any other desired behavior for invalid values
      print(f"Warning: Invalid Sentiment value: {row['Sentiment']}")  # Optional: Print a warning message

  return label_text

dataset['label'] = df.apply(lambda x: to_label_text(x), axis=1) # Applying the updated function

In [ ]:
dataset

,text,label
0,TOP ปันผลระหว่างกาล 1.20 บาท ขึ้น XD วันที่ 12...,<SYMBOL> TOP <ASPECT> ปันผลระหว่างกาล <OPINION...
1,"GPSC เผย ""ทิติพงษ์ จุลพรศิริดี"" ลงจาก CFO ""พรร...",<SYMBOL> GPSC <ASPECT> CFO <OPINION> ลงจาก <NEU>
2,"PTTEP เผย ""สัมฤทธิ์ สำเนียง"" ลงจาก CFO ""ชนมาศ ...",<SYMBOL> PTTEP <ASPECT> CFO <OPINION> ลงจาก <NEU>
3,IIG ยกเลิกเพิ่มทุนขาย Sycamore ชี้ตั้งราคาต่ำไ...,<SYMBOL> IIG <ASPECT> เพิ่มทุน <OPINION> ยกเลิ...
4,IIG ยกเลิกเพิ่มทุนขาย Sycamore ชี้ตั้งราคาต่ำไ...,<SYMBOL> IIG <ASPECT> ผู้ถือหุ้น <OPINION> เสี...
...,...,...
394,NUSA ยันมีแหล่งเงินพอไถ่ถอนหุ้นกู้ / ชุด NMO24...,<SYMBOL> NUSA <ASPECT> เงิน <OPINION> พอ <NEU>
395,JKN เลื่อนส่งงบ Q1/67 ไปอีก รอศาลแต่งตั้งผู้ทำ...,<SYMBOL> JKN <ASPECT> งบ <OPINION> เลื่อน <NEU>
396,ก.ล.ต.เกาะติดหุ้นกู้ EA ‘ThaiBMA’ จับตา 1-2 ส....,<SYMBOL> EA <ASPECT> บริษัท <OPINION> ประชุม <...
397,“เบร็นตัน” ลาออกทุกตำแหน่งบริหาร RML “ปลื้มจิต...,<SYMBOL> RML <ASPECT> บริหาร <OPINION> ลาออก <...


In [ ]:
save_folder = '/content/drive/MyDrive/KMITL/Fourth Year/Project'
save_filename = file_dropdown.value
dataset.to_csv(os.path.join(save_folder, save_filename), index=False)